In [193]:
import pandas as pd
import numpy as np
import re
import re as sub
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

In [196]:
file_path = 'corpus_all.txt'  # Replace with the actual path to your text file

lines = []  # Empty list to store the lines

try:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            lines.append(line.strip())  # Append the line to the list, removing any leading/trailing whitespace
    with open('stopwords-bn.txt', 'r', encoding='utf-8') as test:
        stopwords_bn = test.readlines()
        # the above stopwords contains newline \n
        stop_bn = []

        for word in stopwords_bn:
            stop_bn.append(word.rstrip("\r\n"))
            
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")

#file_path = 'corpus_all.txt'  # Replace with the actual path to your text file
first_words = []  # Empty list to store the first words

try:
    for line in lines:
        words = line.strip().split()  # Split the line into words
        if words:
            first_word = words[0]  # Get the first word
            first_words.append(first_word)  # Append the first word to the list

except FileNotFoundError:
    print("File not found. Please check the file path and try again.")


# Define the status levels
status_levels = ["happy","angry","fear","disgust","sad","surprise",]

# Initialize a list to store the extracted status levels and sentences
extracted_data = []

# Iterate over the texts
for text in lines:
    # Initialize variables to store the extracted status level and sentence
    status = ""
    sentence = ""

    # Iterate over the status levels
    for level in status_levels:
        if level in text:
            # Split the text based on the status level
            split_text = text.split(level, 1)

            # Extract the status level and sentence
            status = level
            sentence = split_text[1].strip()
            break

    # Append the extracted status level and sentence to the list
    extracted_data.append({"Status": status, "Sentence": sentence})
    
df = pd.DataFrame(extracted_data)

In [197]:

def text_pre_processing(text):
    #remove stop words from the text
    text = ' '.join(word for word in text.split() if word not in stop_bn)
    #remove punctuation from the text
    text = re.sub(r'[^\w\s]', '', text)
    #remove numbers from the text
    text = re.sub(r'[0-9]', '', text)
    #remove extra spaces from the text
    text = re.sub(' +', ' ', text)
    return text

df['Sentence'] = df['Sentence'].apply(text_pre_processing)


In [198]:
df

,Status,Sentence
0,sad,বরবর মননয পরধনমনতর গণপরজতনতর বলদশ সরকর মননয পর...
1,sad,তদনত লভ তদনতর হয়ছ মন রজনতক শকতর তদনত পরব সমনয ...
2,angry,দশক মধযম আয়র দশ রপনতর মগর মলকর দশ পরনত হচছ
3,happy,গরবত মসলম সগরম যদধ চর রণবর আললহ ছর নচ শর নরয় ত...
4,angry,রতর বল কসর সকল
...,...,...
6284,surprise,ভরত পরতদন ১৮ ট রপ কইস এনটর
6285,surprise,সবধন দশ
6286,sad,সবধন দশর নগরক হয় পরধন
6287,angry,চদলম দদ পয়স


In [199]:
X = df['Sentence']

In [200]:
X[0]

'বরবর মননয পরধনমনতর গণপরজতনতর বলদশ সরকর মননয পরধনমনতর সবপনর ডজটল বলদশর সবপন বক ললনকর একজন ছতর শত সহসর আশ দশর এক অকততম ভলবস নজক দশর যগয গড তলর এক তবর আকঙকষ ভরত হযছলম নজ হত পরতষঠত দশর অনযতম বশববদযলয যশর বজঞন পরযকত বশববদযলয ডজটল বলদশ গডত ধরণর পরযকত বশববদযলয পরতষঠ নসনদহ দশ পরচলনর বচকষণতর পরচয বহন কর যশর বসর সবপনর পরতষঠন শত শত টক বনযগ দহত উজড কর অতযনত দখর সথ বশববদযলয জবনর সচনলগন অনভব করছ য পওযর আননদ শকষরথদর মনপরন উজড পডলখ কথ বপরত কমলমত শকষরথর সদ দনতপত পরণ নশর হমক শললতহনর গলন ছতরতত বতল হমক নয সমমনত শকষকদর মরত যওয গল কট ফলর হমক পরণ নশর হমক কটকত নযমত ঘটন মননয পরধনমনতর শকষরথর শকষকদর পতমতর মত সমমন একজন চতরথ শরণর করমচরর শকষকদর নযমত ঘটনর শকর ডজটল বলদশর সবপন পক হনদর বহনর মত হন যয চখর ছতর বনর সললতহনর হয পরতবদ বহষকর শকষরথদর ভষ আনদলন মকতযদধর বভষকময গলর কথ সশসনর বলদশ আদরশ বশববদযলয বশববদযলযর পরশসনর পষ সনতরস মদক বযবসয শকষকদর লঞচত কর ছতরদর জবন ছনমন খল ছতরদর লঞচত বরল নজর সথপন সবপনর সনর বল গডর সমপরণ পরপনথ মননয পরধনমনতর শকষক শকষরথর মলত আরতনদ যশরর আকশ বতস পরকমপত বসরর বসর একজন করমচর পরশসনর ছতরছযয অনযয পরতকর আজ

In [203]:
# Map sentiment labels to numerical values
label_map  = {'happy': 0, 'angry': 1, 'fear': 2, 'disgust': 3, 'sad': 4, 'surprise': 5}
y = df['Status'].map(label_map)

In [204]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [205]:
X_train[0]

'বরবর মননয পরধনমনতর গণপরজতনতর বলদশ সরকর মননয পরধনমনতর সবপনর ডজটল বলদশর সবপন বক ললনকর একজন ছতর শত সহসর আশ দশর এক অকততম ভলবস নজক দশর যগয গড তলর এক তবর আকঙকষ ভরত হযছলম নজ হত পরতষঠত দশর অনযতম বশববদযলয যশর বজঞন পরযকত বশববদযলয ডজটল বলদশ গডত ধরণর পরযকত বশববদযলয পরতষঠ নসনদহ দশ পরচলনর বচকষণতর পরচয বহন কর যশর বসর সবপনর পরতষঠন শত শত টক বনযগ দহত উজড কর অতযনত দখর সথ বশববদযলয জবনর সচনলগন অনভব করছ য পওযর আননদ শকষরথদর মনপরন উজড পডলখ কথ বপরত কমলমত শকষরথর সদ দনতপত পরণ নশর হমক শললতহনর গলন ছতরতত বতল হমক নয সমমনত শকষকদর মরত যওয গল কট ফলর হমক পরণ নশর হমক কটকত নযমত ঘটন মননয পরধনমনতর শকষরথর শকষকদর পতমতর মত সমমন একজন চতরথ শরণর করমচরর শকষকদর নযমত ঘটনর শকর ডজটল বলদশর সবপন পক হনদর বহনর মত হন যয চখর ছতর বনর সললতহনর হয পরতবদ বহষকর শকষরথদর ভষ আনদলন মকতযদধর বভষকময গলর কথ সশসনর বলদশ আদরশ বশববদযলয বশববদযলযর পরশসনর পষ সনতরস মদক বযবসয শকষকদর লঞচত কর ছতরদর জবন ছনমন খল ছতরদর লঞচত বরল নজর সথপন সবপনর সনর বল গডর সমপরণ পরপনথ মননয পরধনমনতর শকষক শকষরথর মলত আরতনদ যশরর আকশ বতস পরকমপত বসরর বসর একজন করমচর পরশসনর ছতরছযয অনযয পরতকর আজ

In [206]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # You can adjust the number of features as needed
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()


In [207]:
X_test_tfidf


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [208]:
print("X_train_tfidf.shape:", X_train_tfidf.shape)

X_train_tfidf.shape: (5031, 5000)


In [209]:
# print(X_train, X_test, y_train, y_test)
print(X_train_tfidf.shape, X_test_tfidf.shape)

(5031, 5000) (1258, 5000)


In [210]:
k = 6  # Number of neighbors (you can choose a different value)
knn_classifier = KNeighborsClassifier(n_neighbors=k)
knn_classifier.fit(X_train_tfidf, y_train)


KNeighborsClassifier(n_neighbors=6)

In [211]:
y_pred = knn_classifier.predict(X_test_tfidf)


In [212]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.34340222575516693
              precision    recall  f1-score   support

           0       0.33      0.97      0.49       391
           1       0.61      0.09      0.15       268
           2       0.14      0.02      0.03        66
           3       0.00      0.00      0.00       141
           4       0.56      0.08      0.14       278
           5       0.45      0.04      0.08       114

    accuracy                           0.34      1258
   macro avg       0.35      0.20      0.15      1258
weighted avg       0.40      0.34      0.23      1258

Confusion Matrix:
[[380   3   0   0   4   4]
 [235  23   3   2   4   1]
 [ 62   1   1   0   2   0]
 [133   1   1   0   5   1]
 [246   6   2   1  23   0]
 [102   4   0   0   3   5]]


# Apply SMOTE

In [214]:
# Assuming you have loaded your dataset into a pandas DataFrame named 'df'
# X should contain the features (in this case, the 'Sentence' column) and y should contain the target class labels (the 'Status' column)
X = df['Sentence']
y = df['Status']

# Create a TfidfVectorizer to convert text to numerical features
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Create an instance of the SMOTE class
smote = SMOTE(sampling_strategy='auto', random_state=42)

# Apply SMOTE to generate synthetic samples for the minority class (fear)
X_resampled, y_resampled = smote.fit_resample(X_vectorized, y)

label_map  = {'happy': 0, 'angry': 1, 'fear': 2, 'disgust': 3, 'sad': 4, 'surprise': 5}
y_resampled = y_resampled.map(label_map)

In [215]:
X_resampled = X_resampled.toarray()

In [216]:
#test train split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [217]:

# Create a KNN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = knn_classifier.predict(X_test)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print(classification_report(y_test, y_pred))

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.5730533683289589
              precision    recall  f1-score   support

           0       0.39      0.43      0.41       390
           1       0.92      0.22      0.36       384
           2       0.94      0.93      0.93       374
           3       0.92      0.60      0.73       388
           4       0.41      0.51      0.45       393
           5       0.42      0.78      0.55       357

    accuracy                           0.57      2286
   macro avg       0.67      0.58      0.57      2286
weighted avg       0.67      0.57      0.57      2286

Confusion Matrix:
[[167   1   6   3 110 103]
 [ 65  85   3   6  98 127]
 [  4   0 346   0  13  11]
 [ 46   3   4 234  42  59]
 [ 91   2   5  12 199  84]
 [ 51   1   3   0  23 279]]
